QFM for targetX

In [5]:
import numpy as np
cutoff_dim=5
r=1.5
val=0.2

target_state=[]
for n in range(cutoff_dim):
    target_state.append((1 / np.sqrt(np.cosh(r)) )*( np.sqrt( np.math.factorial( 2*n )) )/( (2**n)*np.math.factorial(n) )*( np.tanh(r)*np.exp( (-1)**(1/2)*( val + np.pi) ) )**n)
    
target_state

[(0.6519938917982901+0j),
 (-0.4089816533192021-0.08290468546673106j),
 (0.3012915747830949+0.1273840346812343j),
 (-0.223078788697243-0.15261641050806268j),
 (0.1594415209817035+0.1641671375975166j)]

Circuit

In [6]:
import numpy as np
import os
import strawberryfields as sf
from strawberryfields import ops
import tensorflow as tf
import warnings
from itertools import combinations
warnings.filterwarnings("ignore")
physical_devices = tf.config.experimental.list_physical_devices('CPU')
tf.config.experimental.set_visible_devices(physical_devices[0])


def init_weights(modes, layers, active_sd=0.1, passive_sd=1):
    # Number of interferometer parameters:
    M = int(modes * (modes - 1)) 

    # TensorFlow variables
    int1_weights = tf.random.normal(shape=[layers, M], stddev=passive_sd)
    r1_weights = tf.random.normal(shape=[layers, modes], stddev=passive_sd)
    int2_weights = tf.random.normal(shape=[layers, M], stddev=passive_sd)
    s_mag_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    s_phase_weights = tf.random.normal(shape=[layers, modes], stddev=passive_sd)
    r2_weights = tf.random.normal(shape=[layers, modes], stddev=passive_sd)
    dr_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    dp_weights = tf.random.normal(shape=[layers, modes], stddev=passive_sd)
    k_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)

    weights = tf.concat([int1_weights, r1_weights, s_mag_weights,s_phase_weights, int2_weights, r2_weights, dr_weights, dp_weights, k_weights], axis=1)
    weights = tf.Variable(weights)

    return weights

def input_qnn_layer(hid,q):
    with tf.name_scope('inputlayer'):
        ops.Dgate(hid[0]) | q[0]
        ops.Dgate(hid[1]) | q[1]
        
        ops.Rgate(hid[0]) | q[0]
        ops.Rgate(hid[1]) | q[1]
        
        ops.Dgate(hid[0]) | q[0]
        ops.Dgate(hid[1]) | q[1]

# Defining standard QNN layers
def qnn_layer(params,layer_number,q):
    with tf.name_scope('layer_{}'.format(layer_number)):
        N = len(q)
        M = int(modes * (modes - 1)) 
        
        int1 = params[:M]
        r1=params[M:M+N]
        sm = params[M+N:M+2*N]
        sp = params[M+2*N:M+3*N]
        int2 = params[M+3*N:2*M+3*N]
        r2=params[2*M+3*N:2*M+4*N]
        dr = params[2*M+4*N:2*M+5*N]
        dp = params[2*M+5*N:2*M+6*N]
        ker = params[2*M+6*N:2*M+7*N]
        
        theta1=int1[:len(int1)//2]
        phi1=int1[len(int1)//2:]
        
        theta2=int2[:len(int2)//2]
        phi2=int2[len(int2)//2:]
        
        
        
        for k, (q1, q2) in enumerate(combinations(q,2)):
            ops.BSgate(theta1[k], phi1[k]) | (q1, q2)

        for i in range(N):
            ops.Rgate(r1[i]) | q[i]

        for i in range(N):
            ops.Sgate(sm[i],sp[i]) | q[i]

        for k, (q1, q2) in enumerate(combinations(q,2)):
            ops.BSgate(theta2[k], phi2[k]) | (q1, q2)

        for i in range(N):
            ops.Rgate(r2[i]) | q[i]

        for i in range(N):
            ops.Dgate(dr[i], dp[i]) | q[i]

        for i in range(N):
            ops.Kgate(ker[i]) | q[i]


modes=2
layers=4
hidden_units=2


eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": cutoff_dim})
qnn = sf.Program(modes)

# initialize QNN weights
weights = init_weights(modes, layers) #TensorFlow weights
num_params = np.prod(weights.shape)   

sf_params = np.arange(num_params).reshape(weights.shape).astype(np.str)
sf_params = np.array([qnn.params(*i) for i in sf_params])

hid_params = np.arange(num_params,num_params+hidden_units).reshape(hidden_units,1).astype(np.str) #to input
hid_params = np.array([qnn.params(*i) for i in hid_params])

with qnn.context as q:
    input_qnn_layer(hid_params,q)
    for k in range(layers):
        qnn_layer(sf_params[k],k, q)

Cost

In [7]:
def cost(weights, val,y):
     
    mapping_wt = {p.name: w for p, w in zip(sf_params.flatten(), tf.reshape(weights, [-1]))} #Assigns sf parameters to tf variable
    mapping_hid={p.name: w for p, w in zip(hid_params, [val,val] )}  #input data
    mapping_wt.update(mapping_hid) #
    
    results = eng.run(qnn, args=mapping_wt)
    ket = results.state.ket()
    fidelity = tf.abs(tf.reduce_sum(tf.math.conj(ket) * target_state)) ** 2
    cost = (fidelity - 1) ** 2
    return cost  

optimizer = tf.keras.optimizers.Adam()

Train

In [8]:
for i in range(200):
    with tf.GradientTape() as tape:
        loss=cost(weights, val , target_state)
        
    gradients=tape.gradient(loss,weights)
    optimizer.apply_gradients(zip([gradients],[weights]))
    
    if eng.run_progs:
        eng.reset()
    print("loss at iteration {} is {}".format(i,loss))

loss at iteration 0 is 0.39792609214782715
loss at iteration 1 is 0.3599778413772583


loss at iteration 2 is 0.3239508271217346
loss at iteration 3 is 0.2899702489376068


loss at iteration 4 is 0.25812190771102905
loss at iteration 5 is 0.22846068441867828


loss at iteration 6 is 0.20100443065166473
loss at iteration 7 is 0.17574508488178253


loss at iteration 8 is 0.15264582633972168
loss at iteration 9 is 0.13165627419948578


...

loss at iteration 191 is 2.5067947717616335e-11
loss at iteration 192 is 6.571099220309407e-11


loss at iteration 193 is 1.0510348147363402e-10
loss at iteration 194 is 1.5370460459962487e-10


loss at iteration 195 is 1.9122126104775816e-10
loss at iteration 196 is 2.3283064365386963e-10


loss at iteration 197 is 2.7063151719630696e-10
loss at iteration 198 is 2.7063151719630696e-10


loss at iteration 199 is 2.865476744773332e-10
